# Initial Tests for the functions used in Board, Game, and the Visualization Classes of the Percolation Module
- comparing quick find and quick union: found that quick union is much more efficient
- later earlier prototype for Class object and different versions of the print board functions

In [ ]:
from numpy import array, zeros
from random import sample, randint, seed
from colorama import Fore, Style

In [2]:
# These two printing functions only work with quick find as they use direct acess to the root dictionary to retrieve the root

def pre_percolated_board(size, space, final_mark, root, root_dict, board):
    """Prints the board one stage before percolation. Highlights the percolation position. Open positions are black. Closed positions are red."""
    position = 0
    print('[[', end='')
    for row in board:    
        for number in row:
            if position == 0:
                pass
            else:
                if position % size == 0:
                    print()
                    print(' [', end='')

            mark = str(int(number)) + '.'
            
            if mark == '0.':
                if position == final_mark:
                    print(Style.BRIGHT + Fore.RED + mark, end='')
                else:
                    print(Fore.RED + mark, end='')
                print(Style.RESET_ALL, end='')
            else:
                print(mark, end='')
            if (position + 1) % size == 0:
                print(']', end='')
                if position + 1 == space:
                    print(']')
            else:
                print(' ', end='')

            position += 1


def percolated_board(size, space, final_mark, root, root_dict, board):
    """Prints the board at percolation. Highlights the percolation position. Full positions are blue. Open positions are black. Closed positions are red."""
    position = 0
    print('[[', end='')
    for row in board:    
        for number in row:
            if position == 0:
                pass
            else:
                if position % size == 0:
                    print()
                    print(' [', end='')

            mark = str(int(number)) + '.'
            
            if root_dict[position] == root:
                if position == final_mark:
                    print(Style.BRIGHT + Fore.BLUE + mark, end='')
                else:
                    print(Fore.BLUE + mark, end='')
                print(Style.RESET_ALL, end='')
            else:
                if mark == '0.':
                    print(Fore.RED + mark, end='')
                    print(Style.RESET_ALL, end='')
                else:
                    print(mark, end='')
            
            if (position + 1) % size == 0:
                print(']', end='')
                if position + 1 == space:
                    print(']')
            else:
                print(' ', end='')

            position += 1
            
# Alternative Printing by building a list and calling a function- used in final model in the Percolation Module
def p_board(size, space, root, root_dict, board):  
    position = 0
    new_board = ['[[']
    for row in board:    
        for number in row:
            if position == 0:
                pass
            else:
                if position%size == 0:
                    new_board.append('\n [')

            mark = str(int(number)) + '.'
            new_board.append(mark)

            if (position + 1)%size == 0:
                new_board.append(']')

                if position+1 == space:
                    new_board.append(']')
            else:
                new_board.append(' ')

            position += 1
    
    return ''.join(new_board)
#     position = 0
#     for mark in a:
#         if (mark == '0') or (mark == '1'):
#             if root_dict[position] == root:
#                 print(Style.BRIGHT + Fore.BLUE + mark, end='')
#                 print(Style.RESET_ALL, end='')
#             else:
#                 print(mark, end='')
#             position += 1
#         else:
#             print(mark, end='')

In [3]:
def generate_roots(n):
    """Generate and return dictionary to keep track of roots for matrix with n elements"""
    roots = dict()
    for i in range(n):
        roots[i] = i
    return roots

def quick_find(p, q, roots):
    """Update dictionary of roots using quick find algorithm setting all numbers with root of the first number to the root of the second number"""
    for num, root in roots.items():
        if root == p:
            roots[num] = q   

def quick_union(root_p, root_q, roots):
    """Update dictionary of roots using quick union algorithm setting the root of the first number to the root of the second number"""
    roots[root_p] = root_q

def find_root(n, roots):
    """Returns the root of a number for the quick union algorithm"""
    while roots[n] != n:
        n = roots[n]
    return n
    
def connected(a, b, roots):
    """Call quick find method to update roots if the numbers have different roots"""
    p = roots[a]
    q = roots[b]
    if p != q:
        quick_find(p, q, roots)
        
def connected_qu(a, b, roots):
    """Call quick union method to update roots if the numbers have different roots"""
    p = find_root(a, roots)
    q = find_root(b, roots)
    if p != q:
        quick_union(p, q, roots)

# Tested at size of 120
# Fastest way to check percolation!

def percolated(size, space, roots):
    """Check if the numbers have percolated"""
    top_row = set()
    bottom_row = set()
    
    for i in range(size):
        top_row.add(roots[i])
    for i in range(space-size, space):
        bottom_row.add(roots[i])
    
    common_roots = top_row.intersection(bottom_row)
    
    if len(common_roots) != 0:
            return True, list(common_roots)[0]
    
    return False, 0

# the best percolation algorithm using find_root function 
def percolated_qu(size, space, roots):
    """Check if the numbers have percolated"""
    top_row = set()
    bottom_row = set()
    
    for i in range(size):
        top_row.add(find_root(i, roots))
    for i in range(space-size, space):
        bottom_row.add(find_root(i, roots))
    
    common_roots = top_row.intersection(bottom_row)
    
    if len(common_roots) != 0:
            return True, list(common_roots)[0]
    
    return False, 0
   
# Second fastest -> performance times close to the set way- about one or two seconds slower 
def percolated_2(size, space, roots):   
    top_row = []
    bottom_row = []
    for i in range(size):
        top_row.append(roots[i])
    for i in range(space - size, space):
        bottom_row.append(roots[i])
    for root in top_row:
        if root in bottom_row:
            return True, root
    return False, 0

# Second slowest -> about one or two seconds longer than second fastest
def percolated_3(size, space, roots):
    top_row = []
    bottom_row = []
    for i in range(size):
        top_row.append(roots[i])
    for i in range(space - size, space):
        bottom_row.append(roots[i])
    for top_root in top_row:
        for bottom_root in bottom_row:
            if top_root == bottom_root:
                return True, top_root
    return False, 0

# Slowest -> almost twice as slow as using sets
def percolated_4(size, space, roots):
    for top_number in range(size):
        top_root = roots[top_number]
        for bottom_number in range(space - size, space):
            if top_root == roots[bottom_number]:
                return True, top_root
            
    return False, 0

In [4]:
import time

def game(size, show_board, connected, percolated):
    board = zeros((size, size))

    space = size ** 2
    top_edge = 0
    bottom_edge = size - 1 

    root_dict = generate_roots(space)
    seed(5)
    marks = sample(range(0,  space), space)

    count = 1
    start = time.time()
    for mark in marks:
        row = (mark) // size
        column = (mark) % size
        position = (row, column)
        previous_board = board.copy()
        board[row, column] = 1


        # Positions on board relative to new mark
        above = row - 1
        below = row + 1
        left = column - 1
        right = column + 1

        # Check the position above for connection
        if above >= top_edge:
            if board[above, column] == 1:
                connected(mark, mark - size, root_dict)

        # Check the position below for connection
        if below <= bottom_edge:
            if board[below, column] == 1:
                connected(mark, mark + size, root_dict)

        # Check the position to the left for connection
        if left >= top_edge:
            if board[row, left] == 1:
                connected(mark, mark - 1, root_dict)

        # Check the position to the right for connection
        if right <= bottom_edge:
            if board[row, right] == 1:
                connected(mark, mark + 1, root_dict)

        # Check for percolation
        done, root = percolated(size, space, root_dict)
        if done is True:
            percolation = count/space
            end = time.time()
            print(f"Elapsed time: {end - start}\n")
            if (size < 50) and (show_board == True):
                
                # Show the board one step before percolation, highlighting the percolation breaking point
                pre_percolated_board(size, space, mark, root, root_dict, previous_board)
                print("\nDone! The percolation threshold is:", str(round(percolation*100, 2))+"%\n")
                
                # Show the percolated board, highlighting the percolation breaking point
                percolated_board(size, space, mark, root, root_dict, board)

            return percolation
 
        else:
            count += 1

There are four algorithms for checking for percolation. 

Below is a test to see which one is the best performing algorithm.

In [5]:
p1 = game(120, True, connected, percolated)
p2 = game(120, True, connected, percolated_2)
p3 = game(120, True, connected, percolated_3)
p4 = game(120, True, connected, percolated_4)
# Check that the percolation threshold is the same, i.e. the seed worked and all were tested on the same sequence of random numbers
print(p1 == p2 == p3 == p4)
# Show the percolation threshold
print("Threshold", p1)

Elapsed time: 5.0107808113098145

Elapsed time: 6.2029640674591064

Elapsed time: 7.507521152496338

Elapsed time: 13.656944036483765

True
Threshold 0.5970833333333333


Adapted the best performing percolated function and the connected function for quick union.

Compare the times to quick find.

In [6]:
for size in [51, 100, 175]:
    print(f"Quick Find: size={size}")
    game(size, True, connected, percolated)
    print(f"Quick Union: size={size}")
    game(size, True, connected_qu, percolated_qu)
    print()

Quick Find: size=51
Elapsed time: 0.18721723556518555

Quick Union: size=51
Elapsed time: 0.044821977615356445


Quick Find: size=100
Elapsed time: 2.345093011856079

Quick Union: size=100
Elapsed time: 0.28791022300720215


Quick Find: size=175
Elapsed time: 22.17283797264099

Quick Union: size=175
Elapsed time: 1.6307129859924316




In [7]:
game(6, True, connected, percolated)

Elapsed time: 0.0005602836608886719

[[1. 1. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 1. 0.]
 [1. 1. 0. 0. 0. 1.]
 [0. 1. 1. 1. 1. 0.]]

Done! The percolation threshold is: 58.33%

[[1. 1. 0. 1. 0. 1.]
 [1. 1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 1. 0.]
 [1. 1. 0. 0. 0. 1.]
 [0. 1. 1. 1. 1. 0.]]


0.5833333333333334

In [8]:
# Note the visualizer does not work on the percolated board because of the how quick union keeps track of roots
game(6, True, connected_qu, percolated_qu)

Elapsed time: 0.0005350112915039062

[[1. 1. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 1. 0.]
 [1. 1. 0. 0. 0. 1.]
 [0. 1. 1. 1. 1. 0.]]

Done! The percolation threshold is: 58.33%

[[1. 1. 0. 1. 0. 1.]
 [1. 1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1. 1.]
 [1. 0. 1. 0. 1. 0.]
 [1. 1. 0. 0. 0. 1.]
 [0. 1. 1. 1. 1. 0.]]


0.5833333333333334

## Below is an earlier version of the game class, followed by my first Percolation Class coding

Early version of the game/board function test and print test

In [9]:
size = int(input("Enter a size: "))
print()

board = zeros((size, size))

print("Initial Board")
print(board)
print()

space = size**2
top_edge = 0
bottom_edge = size-1 

root_dict = generate_roots(space)
print("The Root Dictionary:")
print(root_dict)
print()

# print(top_edge, bottom_edge)

marks = sample(range(0,  space), space)
print("Original random sequence for generating new, open spaces on the board.")
print(marks)
print()

count = 1
for mark in marks:
    print("Number:", mark)
    row = (mark) // size
    column = (mark) % size
    position = (row, column)
    print("Position:", position)
    previous_board = board.copy()
    board[row, column] = 1
#     print(board)
    
    # Positions on board relative to new mark
    above = row - 1
    below = row + 1
    left = column - 1
    right = column + 1
    
    # Check the position above for connection
    if above >= top_edge:
        if board[above, column] == 1:
            print("Bump!")
            connected(mark, mark - size, root_dict)
    
    # Check the position below for connection
    if below <= bottom_edge:
        if board[below, column] == 1:
            print("Bump!")
            connected(mark, mark + size, root_dict)
    
    # Check the position to the left for connection
    if left >= top_edge:
        if board[row, left] == 1:
            print("Bump!")
            connected(mark, mark - 1, root_dict)
    
    # Check the position to the right for connection
    if right <= bottom_edge:
        if board[row, right] == 1:
            print("Bump!")
            connected(mark, mark + 1, root_dict)
    
    # Check for percolation
    done, root = percolated(size, space, root_dict)
    if done is True:
        print("percolated!")
        print()
        break
    else:
        count += 1
        
print("Done! The game is over!")
print()
print("Percolation threshold:", count/space)
print()
        
print("The Root Dictionary:")
print(root_dict)
print()
print("Penultimate Board:")
print(previous_board)
print()
print("Final Board:")
print(board)
print()
print("Ultimate Board (formatted)")
percolated_board(size, space, position, root, root_dict, board)

Enter a size:  3



Initial Board
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

The Root Dictionary:
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8}

Original random sequence for generating new, open spaces on the board.
[2, 4, 8, 1, 7, 5, 0, 6, 3]

Number: 2
Position: (0, 2)
Number: 4
Position: (1, 1)
Number: 8
Position: (2, 2)
Number: 1
Position: (0, 1)
Bump!
Bump!
Number: 7
Position: (2, 1)
Bump!
Bump!
percolated!

Done! The game is over!

Percolation threshold: 0.5555555555555556

The Root Dictionary:
{0: 0, 1: 8, 2: 8, 3: 3, 4: 8, 5: 5, 6: 6, 7: 8, 8: 8}

Penultimate Board:
[[0. 1. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]

Final Board:
[[0. 1. 1.]
 [0. 1. 0.]
 [0. 1. 1.]]

Ultimate Board (formatted)
[[0. 1. 1.]
 [0. 1. 0.]
 [0. 1. 1.]]


The first protype for the Class Percolation. Note that:
- already switched from Numpy arrays to lists
- tells the user that percolation occured but continues until full connection is made
- there is no validation for size or row and column inputs- will get index out of range error

In [10]:
class Percolation:
    
    open_sites = 0
    
    def __init__(self, size):
        self.size = int(size)
        self.space = (self.size)**2
        self.board = [0 for num in range(self.space)]
        self.numbers = [num + 1 for num in range(self.space)]
        self.roots = [num + 1 for num in range(self.space)]
        
    def is_open(self, n):
        return self.board[n-1] == 1
    
    def open_gate(self, n):
        self.board[n-1] = 1
        self.open_sites += 1

    def number_of_open_sites(self):
        return self.open_sites
#         return sum([1 for num in self.board if num == 1])
#         return self.board.count(1)
    
    def is_full(self, n):             
        if self.is_open(n):
            for i in range(self.size):
                if self.find_root(n) == self.find_root(i+1):
                    return True
        return False
       
    def percolates(self):
        for n in range(self.space-self.size, self.space, 1):
            if self.is_full(n + 1) is True:
                print("Percolates!")
                return
        print("Not Yet!")
        
    def connected(self, a, b):
        p = self.find_root(a)
        q = self.find_root(b)
        if p != q:
            self.quick_union(p, q)

            return True
        
        return False
            
    def find_root(self, n):
        while self.roots[n-1] != n:
            n = self.roots[n-1]
        return n
    
    def quick_union(self, p, q):  
        self.roots[p-1] = q
      
    
    def percolated_board(self):  
        print('[[', end='')
        for position, number in enumerate(self.board):
            if position == 0:
                pass
            else:
                if position % self.size == 0:
                    print()
                    print(' [', end='')

            mark = '0.'

            if self.is_full(position + 1):
                print(Fore.BLUE + mark, end='')
                print(Style.RESET_ALL, end='')         

            elif number == 0:
                
                print(Fore.WHITE + mark, end='')
                print(Style.RESET_ALL, end='')

            else:
      
                print(mark, end='')

            if (position + 1) % self.size == 0:
                print(']', end='')
                if position + 1 == self.space:
                    print(']')
            else:
                print(' ', end='')  
                
        print('\n')
      
    
    def print_board(self):
        for mark in self.p_board():
            print(mark, end='')
        print('\n')
   
    def p_board(self):  
        new_board = ['[[']
        for position, number in enumerate(self.board):
            if position == 0:
                pass
            else:
                if position%self.size == 0:
                    new_board.append('\n [')

            if self.is_full(position + 1):
                mark = 'F' + '.'
            else:
                mark = str(int(number)) + '.'

            new_board.append(mark)

            if (position + 1)%self.size == 0:
                new_board.append(']')

                if position+1 == self.space:
                    new_board.append(']')
            else:
                new_board.append(' ')

        return ''.join(new_board)

In [11]:
class Game:
    
    def __init__(self, auto=True):
        self.auto = auto
 
      
    def play_game(self):
        
        size = input("Enter the size: ")
        
        game = Percolation(size)
        
        
        done = False
        while not done:
            if self.auto is True:
                a = randint(1, game.size)
                b = randint(1, game.size)
            else:
                a = int(input("A: "))
                b = int(input("B: "))
            
            position = (a-1)*game.size + b
   
            if game.is_open(position):
                if self.auto is False:
                    print(f"{a} and {b} has already been played")
                continue

            game.open_gate(position)
            print(f"{a} and {b}")

            above = position - game.size
            below = position + game.size
            left = position - 1
            right = position + 1

            if a != 1:
                if game.is_open(above):
                    game.connected(position, above)

            if a != game.size:
                if game.is_open(below):
                    game.connected(position, below)

            if b != 1:
                if game.is_open(left):
                    game.connected(position, left)


            if b != game.size:
                if game.is_open(right):
                    game.connected(position, right)

            game.percolated_board()
            game.percolates()

            if game.number_of_open_sites() == game.space:
                done = True

In [13]:
g = Game()
g.play_game()


Enter the size:  4


3 and 1
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
3 and 4
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
2 and 2
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
3 and 3
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
1 and 3
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
4 and 3
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
2 and 4
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
4 and 2
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
4 and 1
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
1 and 2
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Not Yet!
1 and 4
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Percolates!
2 and 3
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Percolates!
3 and 2
[[0. 0. 0. 0.]
 [0. 0. 0. 